In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'titanic',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
## Keep only numeric columns. Randomly split the dataset into training and test

X = []
for s in a[0].keys():
    if a[0][s].dtype != tf.string:
        X.append(a[0][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[1].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[1].numpy()[test_idx]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    if min(ed1_results['test']['error']) < best_val:
        print("NEW BEST VALUE!")
        best_val = min(ed1_results['test']['error'])
    
    return {'loss': min(ed1_results['test']['error']), 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['binary:logistic']),
    'eval_metric':hp.choice('eval_metric',[['error']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.964562625032131, 'colsample_bytree': 0.893397411136033, 'eta': 0.2988171302307861, 'eval_metric': ('error',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0, 'max_depth': 4, 'min_child_weight': 63.195278159027254, 'objective': 'binary:logistic', 'subsample': 0.743471299161638}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[11:01:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	t

[4]	train-error:0.20415	test-error:0.24173                            

[5]	train-error:0.20415	test-error:0.23919                            

[6]	train-error:0.20415	test-error:0.23664                            

[7]	train-error:0.19105	test-error:0.22901                            

[8]	train-error:0.19105	test-error:0.22901                            

[9]	train-error:0.19214	test-error:0.22901                            

[10]	train-error:0.19105	test-error:0.22901                           

[11]	train-error:0.19105	test-error:0.23155                           

[12]	train-error:0.19214	test-error:0.23155                           

[13]	train-error:0.18887	test-error:0.23155                           

[14]	train-error:0.18887	test-error:0.22392                           

[15]	train-error:0.18668	test-error:0.22901                           

[16]	train-error:0.18668	test-error:0.22901                           

[17]	train-error:0.18668	test-error:0.21883                     

[1]	train-error:0.21070	test-error:0.24173                             

[2]	train-error:0.21070	test-error:0.24173                             

[3]	train-error:0.21397	test-error:0.27226                             

[4]	train-error:0.21397	test-error:0.27226                             

[5]	train-error:0.21397	test-error:0.27226                             

[6]	train-error:0.20415	test-error:0.26718                             

[7]	train-error:0.20415	test-error:0.26718                             

[8]	train-error:0.20415	test-error:0.26718                             

[9]	train-error:0.20415	test-error:0.26718                             

[10]	train-error:0.20415	test-error:0.26718                            

[11]	train-error:0.21397	test-error:0.27226                            

[12]	train-error:0.20415	test-error:0.26718                            

[13]	train-error:0.20306	test-error:0.26463                            

[14]	train-error:0.20306	test-error:0.26463        


[0]	train-error:0.21070	test-error:0.27735                             

[1]	train-error:0.19323	test-error:0.22646                             

[2]	train-error:0.18996	test-error:0.23155                             

[3]	train-error:0.18887	test-error:0.23155                             

[4]	train-error:0.18996	test-error:0.23155                             

[5]	train-error:0.18013	test-error:0.23155                             

[6]	train-error:0.18013	test-error:0.23155                             

[7]	train-error:0.18668	test-error:0.24682                             

[8]	train-error:0.18231	test-error:0.22901                             

[9]	train-error:0.18668	test-error:0.22646                             

[10]	train-error:0.17467	test-error:0.23155                            

[11]	train-error:0.17358	test-error:0.22901                            

[12]	train-error:0.17358	test-error:0.22901                            

[13]	train-error:0.17904	test-error:0.23410       

[0]	train-error:0.20197	test-error:0.25191                             

[1]	train-error:0.19323	test-error:0.21374                             

[2]	train-error:0.19323	test-error:0.21374                             

[3]	train-error:0.18777	test-error:0.21120                             

[4]	train-error:0.18777	test-error:0.21120                             

[5]	train-error:0.18668	test-error:0.23664                             

[6]	train-error:0.17576	test-error:0.22137                             

[7]	train-error:0.16812	test-error:0.22901                             

[8]	train-error:0.16812	test-error:0.22901                             

[9]	train-error:0.16921	test-error:0.22901                             

[10]	train-error:0.17467	test-error:0.22901                            

[11]	train-error:0.17358	test-error:0.22901                            

[12]	train-error:0.17358	test-error:0.22901                            

[13]	train-error:0.16921	test-error:0.22392        

...

[69]	train-error:0.08952	test-error:0.18829                             

[70]	train-error:0.08952	test-error:0.18829                             

[71]	train-error:0.08734	test-error:0.18829                             

[72]	train-error:0.08843	test-error:0.18575                             

[73]	train-error:0.08734	test-error:0.18321                             

[74]	train-error:0.08734	test-error:0.18575                             

[75]	train-error:0.08625	test-error:0.18321                             

[76]	train-error:0.08625	test-error:0.18829                             

[77]	train-error:0.08625	test-error:0.18321                             

[78]	train-error:0.08625	test-error:0.19084                             

[79]	train-error:0.08734	test-error:0.19084                             

[80]	train-error:0.08515	test-error:0.19338                             

[81]	train-error:0.08515	test-error:0.19084                             

[82]	train-error:0.08515	test-error:0.

[64]	train-error:0.37336	test-error:0.40204                             

[65]	train-error:0.37336	test-error:0.40204                             

[66]	train-error:0.37336	test-error:0.40204                             

[67]	train-error:0.37336	test-error:0.40204                             

[68]	train-error:0.37336	test-error:0.40204                             

[69]	train-error:0.37336	test-error:0.40204                             

[70]	train-error:0.37336	test-error:0.40204                             

[71]	train-error:0.37336	test-error:0.40204                             

[72]	train-error:0.37336	test-error:0.40204                             

[73]	train-error:0.37336	test-error:0.40204                             

[74]	train-error:0.37336	test-error:0.40204                             

[75]	train-error:0.37336	test-error:0.40204                             

[76]	train-error:0.37336	test-error:0.40204                             

[77]	train-error:0.37336	test-error:0.

[59]	train-error:0.02838	test-error:0.19084                             

[60]	train-error:0.02838	test-error:0.19084                             

[61]	train-error:0.02838	test-error:0.19338                             

[62]	train-error:0.02838	test-error:0.19338                             

[63]	train-error:0.02729	test-error:0.19593                             

[64]	train-error:0.02729	test-error:0.19338                             

[65]	train-error:0.02838	test-error:0.19847                             

[66]	train-error:0.02729	test-error:0.19084                             

[67]	train-error:0.02729	test-error:0.19084                             

[68]	train-error:0.02729	test-error:0.18829                             

[69]	train-error:0.02729	test-error:0.19338                             

[70]	train-error:0.02729	test-error:0.19084                             

[71]	train-error:0.02729	test-error:0.19084                             

[72]	train-error:0.02729	test-error:0.

[55]	train-error:0.12882	test-error:0.22646                             

[56]	train-error:0.12882	test-error:0.22901                             

[57]	train-error:0.12882	test-error:0.22646                             

[58]	train-error:0.12882	test-error:0.22392                             

[59]	train-error:0.12882	test-error:0.22901                             

[60]	train-error:0.12991	test-error:0.22646                             

[61]	train-error:0.12991	test-error:0.22646                             

[62]	train-error:0.12882	test-error:0.22137                             

[63]	train-error:0.12555	test-error:0.22392                             

[64]	train-error:0.12773	test-error:0.22646                             

[65]	train-error:0.12555	test-error:0.22646                             

[66]	train-error:0.12664	test-error:0.22392                             

[67]	train-error:0.12445	test-error:0.22392                             

[68]	train-error:0.12555	test-error:0.

[50]	train-error:0.16812	test-error:0.22646                             

[51]	train-error:0.16921	test-error:0.22901                             

[52]	train-error:0.16703	test-error:0.22901                             

[53]	train-error:0.17031	test-error:0.22646                             

[54]	train-error:0.17031	test-error:0.22901                             

[55]	train-error:0.16921	test-error:0.22901                             

[56]	train-error:0.16921	test-error:0.22901                             

[57]	train-error:0.16812	test-error:0.22646                             

[58]	train-error:0.16921	test-error:0.22901                             

[59]	train-error:0.16812	test-error:0.23155                             

[60]	train-error:0.17249	test-error:0.22901                             

[61]	train-error:0.17140	test-error:0.22901                             

[62]	train-error:0.16921	test-error:0.23410                             

[63]	train-error:0.16921	test-error:0.

In [4]:
print(best_val)
print(space_eval(spc, best))

0.173028
{'alpha': 0.04060866571133826, 'btype': 'I', 'colsample_bylevel': 0.8525759880376474, 'colsample_bytree': 0.8646165944949851, 'eta': 0.0507146541706267, 'eval_metric': ('error',), 'extra_dims': 0, 'gamma': 2.2052928654195097e-06, 'lambda': 0, 'max_depth': 9, 'min_child_weight': 0.02576575254466821, 'objective': 'binary:logistic', 'subsample': 0.568325016381533}
